In [1]:
import pandas as pd
import numpy as np

ratings = pd.read_csv("../data/ratings.csv")
movies = pd.read_csv("../data/movies.csv")



In [2]:
movies["genres"] = movies["genres"].str.split("|")
movies_exploded = movies.explode("genres")


In [3]:
movie_stats = (
    ratings
    .groupby("movieId")
    .agg(
        avg_rating=("rating", "mean"),
        rating_count=("rating", "count")
    )
    .reset_index()
)


In [4]:
def extract_genres_from_liked_movies(liked_titles):
    liked_movies = movies[movies["title"].isin(liked_titles)]

    if liked_movies.empty:
        return set()

    genres = movies_exploded[
        movies_exploded["movieId"].isin(liked_movies["movieId"])
    ]["genres"]

    return set(genres)


In [5]:
def recommend_movies_genre_and_liked(
    liked_titles=None,
    preferred_genres=None,
    n=10,
    min_ratings=50
):
    liked_titles = liked_titles or []
    preferred_genres = preferred_genres or []

    # 1️⃣ Genres from liked movies
    genres_from_likes = extract_genres_from_liked_movies(liked_titles)

    # 2️⃣ Combine all genres
    final_genres = set(preferred_genres).union(genres_from_likes)

    if not final_genres:
        return "❌ No genres found. Provide liked movies or genres."

    # 3️⃣ Candidate movies
    candidate_movies = movies_exploded[
        movies_exploded["genres"].isin(final_genres)
    ]["movieId"].unique()

    # 4️⃣ Remove already liked movies
    liked_movie_ids = movies[movies["title"].isin(liked_titles)]["movieId"]
    candidate_movies = [m for m in candidate_movies if m not in liked_movie_ids.values]

    # 5️⃣ Rank movies
    recommendations = (
        movie_stats[movie_stats["movieId"].isin(candidate_movies)]
        .query("rating_count >= @min_ratings")
        .sort_values("avg_rating", ascending=False)
        .head(n)
        .merge(movies, on="movieId")
    )

    return recommendations[["title", "genres", "avg_rating", "rating_count"]]


In [9]:
## Example 1️⃣: Liked movies + genres
recommend_movies_genre_and_liked(
    liked_titles=[
        "Inception (2010)",
        "The Dark Knight (2008)"
    ],
    preferred_genres=["Sci-Fi", "Thriller"],
    n=10
)


,title,genres,avg_rating,rating_count
0,"Shawshank Redemption, The (1994)","[Crime, Drama]",4.429022,317
1,"Godfather, The (1972)","[Crime, Drama]",4.289062,192
2,Fight Club (1999),"[Action, Crime, Drama, Thriller]",4.272936,218
3,Cool Hand Luke (1967),[Drama],4.271930,57
4,Rear Window (1954),"[Mystery, Thriller]",4.261905,84
5,"Godfather: Part II, The (1974)","[Crime, Drama]",4.259690,129
6,"Departed, The (2006)","[Crime, Drama, Thriller]",4.252336,107
7,Goodfellas (1990),"[Crime, Drama]",4.250000,126
8,Casablanca (1942),"[Drama, Romance]",4.240000,100
9,"Dark Knight, The (2008)","[Action, Crime, Drama, IMAX]",4.238255,149


In [10]:
## Example 2️⃣: Only liked movies (no genres)
recommend_movies_genre_and_liked(
    liked_titles=[
        "Interstellar (2014)"
    ],
    n=10
)


,title,genres,avg_rating,rating_count
0,"Dark Knight, The (2008)","[Action, Crime, Drama, IMAX]",4.238255,149
1,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Sci-Fi]",4.231076,251
2,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]",4.215640,211
3,"Matrix, The (1999)","[Action, Sci-Fi, Thriller]",4.192446,278
4,Brazil (1985),"[Fantasy, Sci-Fi]",4.177966,59
5,Eternal Sunshine of the Spotless Mind (2004),"[Drama, Romance, Sci-Fi]",4.160305,131
6,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]",4.137755,196
7,Toy Story 3 (2010),"[Adventure, Animation, Children, Comedy, Fanta...",4.109091,55
8,Blade Runner (1982),"[Action, Sci-Fi, Thriller]",4.100806,124
9,Inception (2010),"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",4.066434,143


In [11]:
## Example 3️⃣: Only genres (cold start user)
recommend_movies_genre_and_liked(
    preferred_genres=["Drama", "Romance"],
    n=10
)


,title,genres,avg_rating,rating_count
0,"Shawshank Redemption, The (1994)","[Crime, Drama]",4.429022,317
1,"Godfather, The (1972)","[Crime, Drama]",4.289062,192
2,Fight Club (1999),"[Action, Crime, Drama, Thriller]",4.272936,218
3,Cool Hand Luke (1967),[Drama],4.271930,57
4,"Godfather: Part II, The (1974)","[Crime, Drama]",4.259690,129
5,"Departed, The (2006)","[Crime, Drama, Thriller]",4.252336,107
6,Goodfellas (1990),"[Crime, Drama]",4.250000,126
7,Casablanca (1942),"[Drama, Romance]",4.240000,100
8,"Dark Knight, The (2008)","[Action, Crime, Drama, IMAX]",4.238255,149
9,"Princess Bride, The (1987)","[Action, Adventure, Comedy, Fantasy, Romance]",4.232394,142
